# Working with Agent Jobs with dbatools

You can use dbatools to administer your Agent jobs. Make sure that you have first run your create container notebook

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

The cell block below will create the variables that you will need for the rest of the notebook

You can run the code by using the arrow keys and shift and enter

In [2]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

 Creating connection to the containers
We have a connection to the containers


## List the Agent jobs on the instance

You can very quickly list all the agent jobs on an instance with dbatools

In [3]:
Get-DbaAgentJob -SqlInstance $sql1 | Format-Table


ComputerName InstanceName SqlInstance  Name                                     Category
------------ ------------ -----------  ----                                     --------           
localhost    MSSQLSERVER  f65aa80e71d9 CommandLog Cleanup                       Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 DatabaseBackup - SYSTEM_DATABASES - FULL Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 DatabaseBackup - USER_DATABASES - DIFF   Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 DatabaseBackup - USER_DATABASES - FULL   Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 DatabaseBackup - USER_DATABASES - LOG    Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 DatabaseIntegrityCheck - SYSTEM_DATABAS… Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 DatabaseIntegrityCheck - USER_DATABASES  Database Maintenan…
localhost    MSSQLSERVER  f65aa80e71d9 IndexOptimize - USER_DATABASES           Database Maintenan…
localhost 

You can get info about a particular job

In [5]:
Get-DbaAgentJob -SqlInstance $sql1 -Job 'The Beard is Important'


ComputerName           : localhost
InstanceName           : MSSQLSERVER
SqlInstance            : f65aa80e71d9
Name                   : The Beard is Important
Category               : [Uncategorized (Local)]
OwnerLoginName         : sqladmin
CurrentRunStatus       : Idle
CurrentRunRetryAttempt : 0
Enabled                : True
LastRunDate            : 21/12/2019 14:42:01
LastRunOutcome         : Failed
HasSchedule            : False
OperatorToEmail        : 
CreateDate             : 21/12/2019 14:40:46




Job Steps

In [6]:
Get-DbaAgentJobStep -SqlInstance $SQL1 -Job 'The Beard is Important'


ComputerName   : localhost
InstanceName   : MSSQLSERVER
SqlInstance    : f65aa80e71d9
AgentJob       : The Beard is Important
Name           : Run this SQL
SubSystem      : TransactSql
LastRunDate    : 21/12/2019 14:42:01
LastRunOutcome : Failed
State          : Existing




You can remove jobs 

**PLEASE DON'T DO THIS IN PRODUCTION**

In [10]:
Get-DbaAgentJob -SqlInstance $SQL2 | Remove-DbaAgentJob -Confirm:$false


ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 7150a7015d41
Name         : CommandLog Cleanup
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 7150a7015d41
Name         : DatabaseBackup - SYSTEM_DATABASES - FULL
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 7150a7015d41
Name         : DatabaseBackup - USER_DATABASES - DIFF
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 7150a7015d41
Name         : DatabaseBackup - USER_DATABASES - FULL
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 7150a7015d41
Name         : DatabaseBackup - USER_DATABASES - LOG
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 7150a7015d41
Name         : DatabaseIntegrityCheck - SYSTEM_DATABASES
Status       : Dropped

ComputerName : localhost
InstanceName : MSSQLSERVER
SqlInstance  : 

We have no jobs on SQL2

In [15]:
 $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaAgentJob -SqlInstance $SQL2

If you have run through the notebooks, you might still have the Availability Group that was created in Notebook 03

You would want your Agent Jobs to be the same on all replicas so that if you failover everything works

dbatools has you covered

In [16]:
Copy-DbaAgentJob -Source $SQL1 -Destination $SQL2


Type      Name                                      Status     Notes
----      ----                                      ------     -----
Agent Job CommandLog Cleanup                        Successful 
Agent Job DatabaseBackup - SYSTEM_DATABASES - FULL  Successful 
Agent Job DatabaseBackup - USER_DATABASES - DIFF    Successful 
Agent Job DatabaseBackup - USER_DATABASES - FULL    Successful 
Agent Job DatabaseBackup - USER_DATABASES - LOG     Successful 
Agent Job DatabaseIntegrityCheck - SYSTEM_DATABASES Successful 
Agent Job DatabaseIntegrityCheck - USER_DATABASES   Successful 
Agent Job IndexOptimize - USER_DATABASES            Successful 
Agent Job sp_delete_backuphistory                   Successful 
Agent Job sp_purge_jobhistory                       Successful 
Agent Job The Beard is Important                    Successful 



and just like that all jobs are on SQL2

In [18]:
 $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaAgentJob -SqlInstance $SQL2|Format-Table


ComputerName InstanceName SqlInstance  Name                                     Category
------------ ------------ -----------  ----                                     --------           
localhost    MSSQLSERVER  7150a7015d41 CommandLog Cleanup                       Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 DatabaseBackup - SYSTEM_DATABASES - FULL Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 DatabaseBackup - USER_DATABASES - DIFF   Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 DatabaseBackup - USER_DATABASES - FULL   Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 DatabaseBackup - USER_DATABASES - LOG    Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 DatabaseIntegrityCheck - SYSTEM_DATABAS… Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 DatabaseIntegrityCheck - USER_DATABASES  Database Maintenan…
localhost    MSSQLSERVER  7150a7015d41 IndexOptimize - USER_DATABASES           Database Maintenan…
localhost 

# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!